In [1]:
using JuMP
using DataFrames
using CSV
using Gurobi
using DataFrames
using Random
using LinearAlgebra
using Clustering

include("../LShaped.jl/src/structs.jl")
include("../LShaped.jl/src/setup.jl")
include("../LShaped.jl/src/algorithm.jl")

include("./parameters.jl")
include("./get_functions.jl")
include("./modification_functions_async.jl")

loadcsv = CSV.File("./LOAD.csv");
ptdfdf = DataFrame(CSV.File("./ptdfsmall.csv"));
loaddf = DataFrame(CSV.File("./loaddata.csv"))
winddf = DataFrame(CSV.File("./winddata.csv"))

part1458 = DataFrame(CSV.File("./scenarios/part1458.csv"))

,scenid,date,h1,h2,h3,h4,h5,h6,h7
,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1/1/2002,77.631,130.172,170.516,144.72,32.067,47.563,45.309
2,2,1/2/2002,36.698,26.988,10.85,2.57,0.018,0.004,0.149
3,3,1/3/2002,5.438,37.495,67.072,72.769,95.062,145.275,197.927
4,4,1/4/2002,406.631,406.637,406.64,406.639,406.64,406.64,406.636
5,5,1/5/2002,404.66,307.087,258.291,308.667,404.333,404.224,401.801
6,6,1/6/2002,67.224,108.341,145.046,118.118,127.979,150.923,126.779
7,7,1/7/2002,27.691,4.203,6.271,12.997,26.301,34.522,42.752
8,8,1/8/2002,186.944,113.873,80.828,57.006,48.256,36.013,20.416
9,9,1/9/2002,306.276,305.284,293.739,290.443,283.613,266.227,228.05


,1,2,3,4,5,6,7,8,9,10,11,12
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,4276,5742,1309,6698,2380,1506,3411,4314,6046,3962,5027,3331
2,1255,1776,6310,4467,5131,6530,1461,5377,4140,1485,5671,6497
3,4590,3466,1050,327,3884,2770,2657,3941,5520,5506,6242,6142
4,2658,1381,3786,960,5704,6388,2516,2907,3852,346,1594,2245
5,1056,6853,5795,5064,1204,3835,669,1533,3762,3144,608,5446


In [4]:
let 
    lmax = 0.0
    wmax = 0.0
    for i = 1:24
        #println(i, " ", maximum(loaddf[:,2+i]))
        if maximum(loaddf[:,2+i]) > lmax
            lmax = maximum(loaddf[:,2+i])
        end
        #println(maximum(winddf[:,2+i]))
        if maximum(winddf[:,2+i]) > wmax
            wmax = maximum(winddf[:,2+i])
        end
    end
    global lmax = lmax
    global wmax = wmax
end

loaddis = load_distribution_dict(loadcsv);

ptdfdict = Dict()

for i = 1:38
    br = ptdfdf[i,1]
    ptdfdict[br] = Dict()
    for j = 2:25
        bus = parse(Int64,names(ptdfdf)[j])
        ptdfdict[br][bus] = ptdfdf[i,j]
    end
end

lrts = 2850.0
wrts = 713.5

713.5

In [11]:
loadmat = lrts/lmax*1.35*transpose(Matrix(loaddf[:,3:26]))

24×6940 Matrix{Float64}:
 1528.9   1560.76  1802.91  1743.55  …  1703.15  1737.24  1713.73  2016.6
 1501.41  1544.41  1790.01  1727.34     1665.55  1678.0   1656.98  1972.26
 1479.11  1551.54  1798.57  1727.12     1644.33  1644.63  1618.95  1946.77
 1470.75  1574.01  1824.26  1741.72     1650.61  1634.53  1604.09  1948.26
 1480.52  1624.33  1879.43  1786.16     1685.06  1650.68  1621.34  1984.04
 1510.32  1736.15  2000.2   1892.66  …  1759.41  1707.68  1678.13  2058.58
 1550.43  1895.09  2158.94  2041.58     1862.43  1800.39  1772.92  2162.72
 1576.84  1974.21  2240.12  2111.72     1941.26  1868.34  1859.86  2262.58
 1609.38  1980.68  2181.62  2072.71     1988.04  1914.0   1934.38  2351.8
 1666.91  1984.52  2099.14  2004.01     2022.82  1965.21  2022.04  2438.4
 1702.71  1969.7   2010.54  1932.61  …  2045.81  2004.29  2105.88  2505.16
 1705.36  1910.94  1918.03  1853.34     2063.85  2024.14  2166.37  2535.38
 1677.65  1864.29  1818.28  1775.92     2064.02  2029.52  2192.64  2529.88
 16

In [12]:
windmat = wrts/wmax*transpose(Matrix(winddf[:,3:26]))

24×6940 Matrix{Float64}:
 124.887    59.0368        8.74823  654.156  …  622.575   117.173   368.265
 209.41     43.4162       60.319    654.166     622.578   117.704   450.921
 274.313    17.4546      107.9      654.17      619.732    78.7759  559.312
 232.814     4.13441     117.065    654.169     532.001    79.236   640.667
  51.5869    0.028957    152.928    654.17      448.33    123.252   646.714
  76.5156    0.00643488  233.707    654.17   …  455.048   620.255   647.261
  72.8895    0.239699    318.409    654.164     526.472   621.912   646.664
  76.8084    6.23219     418.856    654.162     622.57    619.068   648.373
  93.2672   14.2002      555.184    653.752     622.199   619.068   651.707
  79.2118    8.97184     472.314    653.752     619.353   616.225   646.748
  66.112    19.3577      544.066    653.713  …  622.203   621.92    633.342
  26.3557   22.7908      527.21     653.741     622.203   621.84    642.47
   9.69898  52.5923      524.256    653.62      622.191   621.60

In [13]:
X = loadmat - windmat

24×6940 Matrix{Float64}:
 1404.01  1501.73  1794.16  1089.4    …  1544.55  1114.67  1596.56  1648.33
 1292.0   1501.0   1729.69  1073.17      1524.24  1055.42  1539.27  1521.34
 1204.79  1534.09  1690.67  1072.95      1520.33  1024.9   1540.18  1387.46
 1237.94  1569.88  1707.19  1087.55      1506.0   1102.52  1524.86  1307.59
 1428.94  1624.3   1726.5   1131.99      1459.0   1202.35  1498.08  1337.33
 1433.8   1736.15  1766.49  1238.49   …  1454.89  1252.63  1057.87  1411.32
 1477.54  1894.85  1840.53  1387.42      1596.04  1273.92  1151.01  1516.06
 1500.03  1967.98  1821.26  1457.56      1720.13  1245.77  1240.79  1614.21
 1516.11  1966.48  1626.44  1418.96      1794.24  1291.81  1315.31  1700.09
 1587.7   1975.55  1626.83  1350.25      1904.17  1345.86  1405.81  1791.65
 1636.6   1950.35  1466.48  1278.9    …  1968.09  1382.09  1483.96  1871.82
 1679.0   1888.15  1390.82  1199.59      1989.86  1401.94  1544.53  1892.91
 1667.95  1811.7   1294.02  1122.3       1979.89  1407.33  1571

In [15]:
X = 1/maximum(X)*X

24×6940 Matrix{Float64}:
 0.368347  0.393983  0.470704  0.285807  …  0.292437  0.418863  0.432446
 0.338962  0.393792  0.453791  0.28155      0.276893  0.403834  0.399128
 0.316082  0.402473  0.443553  0.281493     0.268886  0.404071  0.364005
 0.324778  0.411863  0.447889  0.285322     0.289251  0.400051  0.343051
 0.374886  0.426142  0.452954  0.296982     0.315441  0.393028  0.350853
 0.376164  0.455484  0.463445  0.324923  …  0.328632  0.277537  0.370265
 0.387638  0.49712   0.48287   0.363994     0.334217  0.301971  0.397744
 0.393537  0.516306  0.477814  0.382395     0.326832  0.325525  0.423494
 0.397756  0.515913  0.426702  0.37227      0.33891   0.345076  0.446025
 0.416539  0.518293  0.426805  0.354244     0.353091  0.368819  0.470047
 0.429368  0.51168   0.384735  0.335524  …  0.362596  0.389323  0.491079
 0.440492  0.495363  0.364886  0.314718     0.367803  0.405212  0.496611
 0.437593  0.475305  0.339491  0.294439     0.369218  0.412165  0.492898
 0.429052  0.466441  0.306

In [28]:
X1458_1 = Array{Float64}(undef, size(X,1), 1458)

24×1458 Matrix{Float64}:
 4.00015  3.5963    1.2949    …  3.78827   2.85259  5.00591  3.80158
 5.63129  1.42968   1.5129       2.93399   3.5007   5.22977  3.99106
 3.57825  0.894686  1.96341      2.3686    4.59875  5.17752  5.50299
 2.13754  1.44756   1.90054      3.0804    5.05586  3.68207  6.05914
 1.73341  2.73427   2.74344      2.18156   5.79429  3.67393  7.02021
 2.7945   2.38996   2.22636   …  1.63862   4.84419  3.63152  6.61375
 3.22145  2.31612   1.37227      1.7616    3.21475  4.48871  5.27747
 3.4843   3.85772   2.39704      3.64279   3.18219  4.8695   4.27452
 1.87573  1.91356   3.73729      2.26681   4.14067  4.27642  4.23082
 1.56702  2.46398   3.33344      1.75677   4.45818  4.1924   5.06175
 2.83335  3.32433   2.64939   …  0.985717  4.84271  3.62449  6.58964
 2.84589  4.31559   2.0046       1.25097   4.50594  4.27926  5.29447
 2.25375  3.9927    2.04781      1.65875   5.45368  6.74912  4.84517
 2.57421  4.09798   5.53101      2.07219   3.06616  6.50682  5.4294
 1.2139   

In [73]:
trvec = collect(part1458[2,:])

1458-element Vector{Int64}:
 1255
 1776
 6310
 4467
 5131
 6530
 1461
 5377
 4140
 1485
 5671
 6497
 2388
    ⋮
 5233
 6872
  593
 2833
 1717
 1464
 3094
 5719
 4392
 4455
 3064
 5749

In [74]:
for i = 1:1458
    X1458_1[:,i] = X[:, trvec[i]]
end
X1458_1

24×1458 Matrix{Float64}:
 0.327721  0.155774  0.279299  0.233262  …  0.387733  0.27761   0.37303
 0.306212  0.153897  0.263355  0.253368     0.383832  0.258196  0.370445
 0.294725  0.161452  0.253904  0.285505     0.379495  0.246373  0.384999
 0.290704  0.224001  0.25607   0.272564     0.390238  0.240234  0.39799
 0.309898  0.291828  0.277247  0.288748     0.40988   0.253937  0.409268
 0.33461   0.308455  0.317995  0.378689  …  0.450839  0.270318  0.456072
 0.355004  0.329811  0.33085   0.440706     0.462037  0.266934  0.482935
 0.395462  0.346565  0.355321  0.445351     0.442275  0.299258  0.506522
 0.440856  0.369535  0.376776  0.432127     0.444518  0.332379  0.507353
 0.46017   0.381849  0.382543  0.382958     0.365866  0.363595  0.521502
 0.511391  0.384571  0.371725  0.360602  …  0.331091  0.390299  0.559453
 0.566828  0.376744  0.377985  0.36548      0.323779  0.416808  0.592488
 0.596834  0.357926  0.382113  0.35518      0.312286  0.438629  0.61634
 0.641906  0.333357  0.384539

In [80]:
C = Clustering.kmeans(X1458_1,24)

KmeansResult{Matrix{Float64}, Float64, Int64}([0.4996340662430581 0.26979099932589673 … 0.2301499690030238 0.2931562276014527; 0.4747495409215031 0.2648848964087668 … 0.22018236076047773 0.27594451427098105; … ; 0.5745280064944674 0.27641706583262793 … 0.30317949709268527 0.38714936898675567; 0.5280331542772808 0.24940929197736725 … 0.2747873912456607 0.3493910811996967], [6, 7, 21, 7, 16, 21, 2, 20, 14, 11  …  17, 3, 9, 23, 6, 12, 11, 8, 13, 17], [0.014985901650502953, 0.07442419424071378, 0.035775408759131544, 0.05863887545603852, 0.11232671851500697, 0.09052932031541161, 0.04688110249043831, 0.025035434830051884, 0.034614142698695005, 0.009112238828535624  …  0.021735088956921444, 0.0731508501821665, 0.02468099453282946, 0.027788961674398927, 0.016163137141532502, 0.04035772711400831, 0.0820526953700984, 0.042450835664800124, 0.01299579739136636, 0.03983920530678908], [70, 69, 56, 98, 65, 36, 72, 33, 84, 23  …  42, 41, 93, 34, 64, 80, 82, 48, 68, 89], [70, 69, 56, 98, 65, 36, 72, 33

In [93]:
fieldnames(typeof(C))

(:centers, :assignments, :costs, :counts, :wcounts, :totalcost, :iterations, :converged)

In [81]:
cldict = Dict()
for i = 1:24
    cldict[i] = Int64[]
end
    

In [82]:
for i = 1:1458
    push!(cldict[C.assignments[i]], trvec[i])
end

In [83]:
CSV.write("./scenarios/cldict_24_2.csv",cldict)

"./scenarios/cldict_24_2.csv"

In [84]:
show(C.counts)

[70, 69, 56, 98, 65, 36, 72, 33, 84, 23, 70, 54, 63, 24, 42, 41, 93, 34, 64, 80, 82, 48, 68, 89]

In [90]:
CSV.write("./scenarios/centers_24_2.csv", DataFrame(C.centers))

"./scenarios/centers_24_2.csv"

In [92]:
for test in [420, 69]
    println(test)
    show(C.counts)
end

420
[70, 69, 56, 98, 65, 36, 72, 33, 84, 23, 70, 54, 63, 24, 42, 41, 93, 34, 64, 80, 82, 48, 68, 89]69
[70, 69, 56, 98, 65, 36, 72, 33, 84, 23, 70, 54, 63, 24, 42, 41, 93, 34, 64, 80, 82, 48, 68, 89]

In [95]:
X1458 = Array{Float64}(undef, size(X,1), 1458)

for exp = 1:5
    
    trvec = collect(part1458[exp,:])
    
    for i = 1:1458
        X1458[:,i] = X[:, trvec[i]]
    end

    for K in [12,24]
        C = Clustering.kmeans(X1458,K)
        
        cldict = Dict()
        for i = 1:K
            cldict[i] = Int64[]
        end
        for i = 1:1458
            push!(cldict[C.assignments[i]], trvec[i])
        end
        
        CSV.write("./scenarios/cldict_$(K)_$(exp).csv",cldict)
        CSV.write("./scenarios/centers_$(K)_$(exp).csv", DataFrame(C.centers))
        
        println(K, " ", exp)
        show(C.counts)
        println()
    end
end
        
        
        
    

12 1
[154, 149, 130, 142, 183, 35, 185, 133, 89, 90, 48, 120]
24 1
[54, 84, 78, 64, 37, 72, 61, 71, 70, 66, 31, 28, 106, 29, 105, 79, 58, 66, 64, 64, 46, 36, 51, 38]
12 2
[111, 88, 115, 184, 151, 22, 143, 107, 167, 155, 132, 83]
24 2
[78, 19, 61, 85, 88, 66, 9, 61, 79, 56, 70, 62, 110, 24, 64, 67, 39, 59, 68, 58, 93, 31, 56, 55]
12 3
[183, 115, 115, 126, 173, 88, 84, 138, 103, 29, 155, 149]
24 3
[47, 81, 67, 43, 115, 30, 83, 74, 23, 40, 50, 110, 21, 91, 46, 52, 63, 44, 59, 46, 58, 68, 46, 101]
12 4
[192, 57, 98, 122, 97, 157, 74, 127, 99, 168, 140, 127]
24 4
[64, 86, 62, 42, 82, 54, 83, 58, 35, 33, 91, 81, 53, 67, 59, 55, 88, 4, 62, 81, 54, 58, 64, 42]
12 5
[49, 81, 104, 103, 75, 177, 211, 124, 130, 124, 131, 149]
24 5
[87, 73, 26, 87, 77, 77, 67, 50, 72, 79, 50, 74, 32, 74, 85, 38, 4, 62, 22, 85, 69, 89, 15, 64]
